In [1]:
with open("./input.txt", "r") as file: 
    data = file.read().strip()
    
with open("./sample.txt", "r") as file: 
    sample = file.read().strip()

# Part 1

In [2]:
class Board: 
    def __init__(self, numbers): 
        self.numbers = numbers
        self.draws = [[False for c in row] for row in numbers]
        
    def wins(self) -> bool:
        """
        Checks if any row or any column is complete
        """
        if any(all(row) for row in self.draws): 
            return True
        if any(all(col) for col in zip(*self.draws)):
            return True
        return False
    
    def score(self) -> int: 
        """
        Returns the sum of all numbers not drawn
        """
        score = 0
        for i, row in enumerate(self.numbers):
            for j, value in enumerate(row): 
                if self.draws[i][j] is False: 
                    score += value
        return score
    
    def handle(self, number:int): 
        """
        Checks if the number is in the board, and if yes, 
        marks its position as drawn
        """
        for i, row in enumerate(self.numbers): 
            for j, value in enumerate(row): 
                if value == number: 
                    self.draws[i][j] = True
        return
                
    @classmethod
    def parse(cls, data:str):
        """
        Parses an input string
        """
        numbers = [[int(number) for number in row.strip().split()] 
                   for row in data.strip().split("\n")]
        return cls(numbers)
    
class Game: 
    def __init__(self, boards:list): 
        self.boards = boards
        
    def play(self, numbers:list[int]): 
        for n, number in enumerate(numbers): 
            for board in self.boards: 
                board.handle(number)
                if board.wins():
                    return n, board, board.score() * number
        raise Exception("No board won the game")

In [3]:
def run(inputs:str): 
    #parse the inputs
    numbers = [int(n) for n in inputs.split("\n")[0].strip().split(",")]
    boards  = "\n".join(inputs.split("\n")[1:]).split("\n\n")
    
    #play the game
    rounds, board, score = Game(
        [Board.parse(board) for board in boards]
    ).play(numbers)
    
    print(f"{board} won after {rounds} rounds with {board.score()} x {numbers[rounds]} = {score} points")
    
run(data)
run(sample)

<__main__.Board object at 0x7fc31b0fb9a0> won after 20 rounds with 829 x 17 = 14093 points
<__main__.Board object at 0x7fc31b0f8b50> won after 11 rounds with 188 x 24 = 4512 points


# Part 2

In [4]:
class AltGame(Game): 
    def play(self, numbers:list[int]): 
        for number in numbers:
            for board in self.boards: 
                board.handle(number)
                
            #if there is only 1 board left, play the original game with only that board
            #to get its winning score
            if len([b for b in self.boards if not b.wins()]) == 1: 
                return Game([b for b in self.boards if not b.wins()]).play(numbers)
                
        raise Exception("No board was ever the last to win the game")

In [5]:
def run(inputs:str): 
    #parse the inputs
    numbers = [int(n) for n in inputs.split("\n")[0].strip().split(",")]
    boards  = "\n".join(inputs.split("\n")[1:]).split("\n\n")
    
    #play the game
    rounds, board, score = AltGame([Board.parse(board) for board in boards]).play(numbers)
    
    print(f"{board} was the last to win, after {rounds} rounds with {board.score()} x {numbers[rounds]} = {score} points")
    
run(sample)
run(data)

<__main__.Board object at 0x7fc31b0f8040> was the last to win, after 14 rounds with 148 x 13 = 1924 points
<__main__.Board object at 0x7fc31b10fac0> was the last to win, after 83 rounds with 483 x 36 = 17388 points
